In [96]:
#########################################
# Test functions and non-permanant code #
# Author: Jarrett Reimers               #
# Keep as a reference, do not edit      #
#########################################
import pandas as pd
import os
import datetime
from query import Query

In [98]:
data = pd.read_csv("data/2015/5/1.csv")

starttimes = []
for starttime in data['starttime']:
  starttimes.append(datetime.datetime.strptime(starttime, "%m/%d/%Y %H:%M:%S"))
data['starttime'] = starttimes

data.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,415,2015-05-01 00:00:11,5/1/2015 00:07:06,477,W 41 St & 8 Ave,40.756405,-73.990026,442,W 27 St & 7 Ave,40.746647,-73.993915,17012,Subscriber,1981.0,1
1,1523,2015-05-01 00:00:28,5/1/2015 00:25:52,293,Lafayette St & E 8 St,40.730287,-73.990765,324,DeKalb Ave & Hudson Ave,40.689888,-73.981013,17390,Customer,NaN,0
2,642,2015-05-01 00:00:52,5/1/2015 00:11:35,380,W 4 St & 7 Ave S,40.734011,-74.002939,507,E 25 St & 2 Ave,40.739126,-73.979738,15003,Subscriber,1990.0,1
3,367,2015-05-01 00:00:48,5/1/2015 00:06:55,537,Lexington Ave & E 24 St,40.740259,-73.984092,280,E 10 St & 5 Ave,40.733320,-73.995101,14788,Subscriber,1978.0,1
4,2734,2015-05-01 00:00:51,5/1/2015 00:46:26,426,West St & Chambers St,40.717548,-74.013221,327,Vesey Pl & River Terrace,40.715338,-74.016584,21068,Subscriber,1956.0,2


In [100]:
query = Query()
start_time = query.get_datetime(2015, 5, 13, 0, 0, 0)
end_time = query.get_datetime(2015, 5, 16, 0, 0, 0)
three_day = query.select_time(data, start_time, end_time)
three_day.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
364407,298,2015-05-13 00:00:04,5/13/2015 00:05:03,357,E 11 St & Broadway,40.732618,-73.991580,461,E 20 St & 2 Ave,40.735877,-73.982050,21195,Subscriber,1990.0,1
364408,482,2015-05-13 00:00:13,5/13/2015 00:08:15,493,W 45 St & 6 Ave,40.756800,-73.982912,519,Pershing Square North,40.751873,-73.977706,17503,Customer,NaN,0
364409,132,2015-05-13 00:00:26,5/13/2015 00:02:39,432,E 7 St & Avenue A,40.726218,-73.983799,236,St Marks Pl & 2 Ave,40.728419,-73.987140,15975,Subscriber,1993.0,1
364410,119,2015-05-13 00:00:22,5/13/2015 00:02:21,515,W 43 St & 10 Ave,40.760094,-73.994618,495,W 47 St & 10 Ave,40.762699,-73.993012,18659,Subscriber,1987.0,1
364411,1017,2015-05-13 00:00:29,5/13/2015 00:17:26,488,W 39 St & 9 Ave,40.756458,-73.993722,426,West St & Chambers St,40.717548,-74.013221,19100,Subscriber,1979.0,1


,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
364407,298,2015-05-13 00:00:04,5/13/2015 00:05:03,357,E 11 St & Broadway,40.732618,-73.991580,461,E 20 St & 2 Ave,40.735877,-73.982050,21195,Subscriber,1990.0,1
364408,482,2015-05-13 00:00:13,5/13/2015 00:08:15,493,W 45 St & 6 Ave,40.756800,-73.982912,519,Pershing Square North,40.751873,-73.977706,17503,Customer,NaN,0
364409,132,2015-05-13 00:00:26,5/13/2015 00:02:39,432,E 7 St & Avenue A,40.726218,-73.983799,236,St Marks Pl & 2 Ave,40.728419,-73.987140,15975,Subscriber,1993.0,1
364410,119,2015-05-13 00:00:22,5/13/2015 00:02:21,515,W 43 St & 10 Ave,40.760094,-73.994618,495,W 47 St & 10 Ave,40.762699,-73.993012,18659,Subscriber,1987.0,1
364411,1017,2015-05-13 00:00:29,5/13/2015 00:17:26,488,W 39 St & 9 Ave,40.756458,-73.993722,426,West St & Chambers St,40.717548,-74.013221,19100,Subscriber,1979.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
473604,448,2015-05-15 23:59:30,5/16/2015 00:06:59,280,E 10 St & 5 Ave,40.733320,-73.995101,504,1 Ave & E 15 St,40.732219,-73.981656,18790,Subscriber,1984.0,1
473605,1615,2015-05-15 23:59:31,5/16/2015 00:26:27,348,W Broadway & Spring St,40.724910,-74.001547,471,Grand St & Havemeyer St,40.712868,-73.956981,15885,Subscriber,1985.0,1
473606,783,2015-05-15 23:59:34,5/16/2015 00:12:38,2022,E 59 St & Sutton Pl,40.758491,-73.959206,449,W 52 St & 9 Ave,40.764618,-73.987895,17452,Subscriber,1979.0,1
473607,1870,2015-05-15 23:59:49,5/16/2015 00:30:59,307,Canal St & Rutgers St,40.714275,-73.989900,396,Lefferts Pl & Franklin Ave,40.680342,-73.955769,21646,Subscriber,1986.0,1


In [91]:
year = 2015
month = 5
day = 3
hour = 0
minute = 0
second = 0

data.loc[data['starttime'] <= datetime.datetime(year, month, day, hour, minute, second)].loc[datetime.datetime(year, month, day-1, hour, minute, second) <= data['starttime']]

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
27819,324,2015-05-02 00:00:08,5/2/2015 00:05:32,465,Broadway & W 41 St,40.755136,-73.986580,515,W 43 St & 10 Ave,40.760094,-73.994618,18325,Subscriber,1986.0,1
27820,1821,2015-05-02 00:00:20,5/2/2015 00:30:41,321,Cadman Plaza E & Red Cross Pl,40.699918,-73.989718,349,Rivington St & Ridge St,40.718502,-73.983299,21502,Subscriber,1971.0,1
27821,234,2015-05-02 00:00:21,5/2/2015 00:04:15,483,E 12 St & 3 Ave,40.732233,-73.988900,461,E 20 St & 2 Ave,40.735877,-73.982050,18123,Subscriber,1987.0,1
27822,1100,2015-05-02 00:00:46,5/2/2015 00:19:07,369,Washington Pl & 6 Ave,40.732241,-74.000264,356,Bialystoker Pl & Delancey St,40.716226,-73.982612,16592,Subscriber,1964.0,1
27823,1146,2015-05-02 00:00:50,5/2/2015 00:19:56,445,E 10 St & Avenue A,40.727408,-73.981420,463,9 Ave & W 16 St,40.742065,-74.004432,16074,Subscriber,1979.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53662,548,2015-05-02 23:57:43,5/3/2015 00:06:51,387,Centre St & Chambers St,40.712733,-74.004607,307,Canal St & Rutgers St,40.714275,-73.989900,18322,Subscriber,1971.0,1
53663,487,2015-05-02 23:58:40,5/3/2015 00:06:47,357,E 11 St & Broadway,40.732618,-73.991580,236,St Marks Pl & 2 Ave,40.728419,-73.987140,16627,Subscriber,1970.0,1
53664,616,2015-05-02 23:59:39,5/3/2015 00:09:55,383,Greenwich Ave & Charles St,40.735238,-74.000271,265,Stanton St & Chrystie St,40.722293,-73.991475,16074,Subscriber,1965.0,1
53665,601,2015-05-02 23:59:41,5/3/2015 00:09:42,516,E 47 St & 1 Ave,40.752069,-73.967844,477,W 41 St & 8 Ave,40.756405,-73.990026,15209,Subscriber,1987.0,1
